In [3]:
%config Completer.use_jedi = False

import pickle

import numpy as np
import pandas as pd

pd.set_option('mode.chained_assignment', None)

import util.explore as explore_util

import statsmodels.api as sm

from econml.dml import NonParamDML
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_auc_score, roc_curve

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [4]:
import matplotlib.pyplot as plt

Steps:

1. Code for sector
2. Load in MDG panel
3. Construct non-lag project - MDG DF for sector, with vars as in specifications
4. Extract further project features: size relative to GDP, ratings, basic features of PDO and descriptions, etc.
5. Do some crawls over lag periods, features and feature combinations
6. Get ready for some graph work


Note: for completion stuff, get hold of:

Implementation Completion and Results Report
Implementation Completion Report Review

## Load in all projects from WB project panel, and merge with MDG panel data

In [ ]:
# all_projects = pd.read_csv('../data/clean_wb_proj_all.csv', index_col=0)

In [ ]:
reassemble_proj_country_df = False

df = pd.read_json("../data/aggregated_proj.json", orient="index")
country_panel = pd.read_csv('../data/countrypanel.csv')

if reassemble_proj_country_df:
    df['boardapprovaldate'] = pd.to_datetime(df['boardapprovaldate'])
    df['closingdate'] = pd.to_datetime(df['closingdate'])
    df['closingyear'] = df.closingdate.dt.year


    new_country_df = pd.read_csv('../data/tmp/project_countries.csv')

    ndf = df.merge(new_country_df[['project_country', 'panel_country']], 
                   left_on='countryname', right_on='project_country', how='left')

    ndf = ndf.drop(columns=['countryname'])
    ndf = ndf[ndf.panel_country.notna()]
    
    save_transformed_df = False
    if save_transformed_df:
        ndf.to_csv('../data/transformed_data/projects_with_ccodes.csv')

else:
    ndf = pd.read_csv('../data/transformed_data/projects_with_ccodes.csv', index_col=0, low_memory=False)
    ndf['boardapprovaldate'] = pd.to_datetime(ndf['boardapprovaldate'])
    ndf['closingdate'] = pd.to_datetime(ndf['closingdate'])

In [ ]:
ipf_data = pd.read_csv('../data/prior_proj_paper/ipf_paper_data.csv')
print(ndf.id.isin(ipf_data.projid).sum())

ipf_feature_cols = [
    'origcommamt',
    'cpia_approval',
    'region',
    'gp',
    'fundingsource',
    'fcsatappfy',
    'uppermiddle_income_appfy',
    'origprojlength',
    'prepttl_exp',
    'prepttl_quality_va'    
]

ndf = ndf.merge(ipf_data[['projid'] + ipf_feature_cols], left_on='id', right_on='projid', how='left')

What is:
* mjthemecode

Also, these have XML / extractable information:
* indicatormappingdata

In [ ]:
with open("../data/transformed_data/title_pdo_embeds_reduced.pkl", "rb") as fin:
    embeddings = pickle.load(fin)

em_ext = dict(
    project_id=embeddings["project_ids"],
    embed_x=embeddings["tsne"][:, 0],
    embed_y=embeddings["tsne"][:, 1]
)
embed_df = pd.DataFrame(em_ext)

ndf = ndf.merge(embed_df, left_on="id", right_on="project_id", how="left")

In [ ]:
dli_df = pd.read_csv('../data/transformed_data/clean_dli_pdo_tsne_sector.csv')
dli_df = dli_df.rename(columns={ 'tsne_x': 'dli_x', 'tsne_y': 'dli_y' })

ndf = ndf.merge(dli_df[['id', 'dli_x', 'dli_y']], how='left')

sector_df = pd.read_csv('../data/prior_proj_paper/WB_project_sectors.csv').rename(columns={ 'proj_id': 'id' })
main_sector_df = sector_df[sector_df.flag_main_sector == 1]
main_sector_df.head()

ndf = ndf.merge(main_sector_df[['id', 'sector_code', 'sector_percentage', 'parent_sector_name']], how='left')

ndf['dli_x'] = ndf['dli_x'].fillna(0)
ndf['dli_y'] = ndf['dli_y'].fillna(0)

### Primary data complete, now extract the features we will work with

In [ ]:
basic_features = [
     'id', 
     'regionname', 
     'project_name', 
     'pdo', 
     'boardapprovaldate', 
     'closingdate', 
     'closingyear', 
     'project_country', 
     'panel_country',
]

loan_features = [
    'projectfinancialtype',
    'lendinginstr',
    'sector_percentage' # = percentage in primary sector
] + ipf_feature_cols

# not using combined practice code as limited in reach )recent projects) and seems fairly concentrated
sector_features = [
    'sector1', 'sector2', 'sector3', 'sector4', 'sector5', 'theme1', 'theme2'
]

cat_features = [
    'impagency',
    'cons_serv_reqd_ind',
    'supplementprojectflg',
    'prodlinetext',
    'parent_sector_name'
]

embed_features = [
    'embed_x',
    'embed_y',
    'dli_x',
    'dli_y'
]

wdf = ndf[basic_features + loan_features + sector_features + cat_features + embed_features].fillna(0)
wdf['all_sectors_theme_words'] = wdf[sector_features].apply(lambda row: ' '.join(row.values.astype(str)), axis=1).str.lower()

## Sector coding


* Code projects as health if one of their sectors/themes includes health (and same for education). This seems to pick up more than using the main sector or other sector columns
* For WASH: use water, sanitation, etc

In [ ]:
wdf['is_health_project'] = wdf.all_sectors_theme_words.str.contains('health')
wdf['is_education_project'] = wdf.all_sectors_theme_words.str.contains('edu')

print("Health: " , wdf.is_health_project.value_counts())
print("Education: ", wdf.is_education_project.value_counts())

In [ ]:
data = wdf.merge(country_panel.drop(columns=['regionname']), 
                                    left_on=['panel_country', 'closingyear'], right_on=['countryname', 'year'])

data = data.drop(columns=['countryname', 'year'])
data = data[data.closingyear.notna()]
data['pdo_length'] = data['pdo'].str.len().fillna(0)

data = data.rename(columns = { 'project_country': 'country', 'closingyear': 'year' })
data = explore_util.lag_variable_simple(data, 'gdp_pc_ppp', -5)

In [ ]:
[col for col in data.columns if 'mortality' in col]

We do a negative lag to obtain the value in the future (here just doing gdp pc)

In [ ]:
def sum_feature_imp(category_name, feature_imp, exp_features):
    return sum([feature_imp[col] for col in exp_features if col.startswith(category_name)])

def extract_feature_imp(est, orig_features, exp_features):
    feature_imp = { col: est.feature_importances_[i] for i, col in enumerate(exp_features) }
    summed_feature_imp = { col: sum_feature_imp(col, feature_imp, exp_features) for col in orig_features }
    return feature_imp, summed_feature_imp

def fit_score_model(X, y, est, classification=False):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y if classification else None)
    
    est.fit(X_train, y_train)
    scores = { 'default_score': est.score(X_test, y_test) }
    if classification:
        true_pred = est.predict_proba(X_test)[:, 1]
        scores['fscore_etc'] = precision_recall_fscore_support(y_test, est.predict(X_test), average="binary")
        scores['roc_auc'] = roc_auc_score(y_test, true_pred)
        scores['roc_curve'] = roc_curve(y_test, true_pred)
    else:
        scores['mape'] = mean_absolute_percentage_error(y_test, est.predict(X_test))
    test_data = { "X_test": X_test, "y_test": y_test }
    return est, scores

In [ ]:
trivial_imp_features = [
    'cons_serv_reqd_ind', 
    'impagency',
    'prodlinetext',
    'supplementprojectflg'
]

In [ ]:
health_target = 'mortality_under5_lag-5'

health_to_lag = {
    'mortality_under5': -5,
    'hiv_prevalence': -5,
    'conflict': -5
}

health_observed_X_cols = [
#     'mortality_under5', # = lag_mortality_under5 (since now evaluating at closure year)
    'gdp_pc_ppp',
    'fertility',
    'population',
    'physicians_rate',
    'female_adult_literacy',
    'access_water',
    'access_sanitation',
    'hiv_prevalence_lag-5'
]

probe_feature_cols = loan_features + cat_features + ['pdo_length'] + embed_features
probe_feature_cols = [col for col in probe_feature_cols if col not in trivial_imp_features]

health_results = end_to_end_project_eval(
    data, "health", "mortality_under5_lag-5", health_to_lag,
    observed_X_cols=health_observed_X_cols, 
    loan_feature_cols=probe_feature_cols,
    inverted_outcome=True
)

print("Results for health check: ", health_results["classifier_scores"])
print({ key: round(value, 3) for key, value in health_results["summed_importances"].items() })

In [ ]:
clf_fpr, clf_tpr, _ = health_results["classifier_scores"]["roc_curve"]
plt.plot(clf_fpr, clf_tpr, marker='.', label='Model')
# len(health_results["classifier_scores"]["roc_curve"])

In [ ]:
# get the education projects only
# extract further project features
# run through catboost, econml and the like

In [ ]:
def end_to_end_project_eval(all_data, sector_key_word, target_col, variables_to_lag, observed_X_cols, loan_feature_cols, 
                            regressor=RandomForestRegressor, classifier=RandomForestClassifier, inverted_outcome=False):
    sector_data = all_data.copy()
    
    for var in variables_to_lag:
        sector_data = explore_util.lag_variable_simple(sector_data, var, variables_to_lag[var])
    
    sector_data['is_sector_project'] = sector_data.all_sectors_theme_words.str.contains(sector_key_word)
    sector_data = sector_data[sector_data.is_sector_project]
    print("Sector projects data: ", len(sector_data), " versus all projects: ", len(all_data))
    
    sdata = sector_data[['id'] + observed_X_cols + [target_col]]
    sdata = sdata.dropna()
    print("Clean observations: ", len(sdata))

    # print("Pre scaling: ", sdata[observed_X_cols[:2]].describe())
    observation_scaler = StandardScaler()
    sdata[observed_X_cols] = observation_scaler.fit_transform(sdata[observed_X_cols])
    # print("Shape of endog: ", sdata[target_col].shape, " and exog: ", sm.add_constant(sdata[observed_X_cols]).shape)
    res_est = sm.OLS(endog=sdata[target_col], exog=sm.add_constant(sdata[observed_X_cols])).fit()
    print("Naive R squared of partialling out phase: ", res_est.rsquared, " and f_p: ", res_est.f_pvalue)
    # print("Post scaling: ", sdata[observed_X_cols[:2]].describe())
    
    target_resid = f"residual_target"
    sdata[target_resid] = res_est.resid
    
    forest_data = sdata[['id', target_resid]].merge(all_data[['id'] + loan_feature_cols], how='inner')
#     print(forest_data.isna().sum())
    pre_scale_target_desc = forest_data[target_resid].describe()
    print("Descriptive stats for target: ", pre_scale_target_desc)
    
    numeric_cols = forest_data.select_dtypes(include=np.number).columns.tolist()
    treatment_scaler = StandardScaler()
    forest_data[numeric_cols] = treatment_scaler.fit_transform(forest_data[numeric_cols])

    categorical_cols = [col for col in loan_feature_cols if col not in numeric_cols]
    forest_data = pd.get_dummies(forest_data, columns=categorical_cols)

    forest_data = forest_data.dropna()
    print("Clean within project characteristics: ", len(forest_data))
    
    pos_std_dev_threshold = 0.1
    forest_data[f'{target_resid}_above_threshold'] = (
        forest_data[target_resid] > pos_std_dev_threshold if not inverted_outcome else 
            forest_data[target_resid] < pos_std_dev_threshold
    )
    print("Projects with residual above mean: ", len(forest_data[forest_data[target_resid] > 0]))
    print("Projects with positive residual above threshold: ", len(forest_data[forest_data[target_resid] > pos_std_dev_threshold]))
    
    nreg = regressor()
    nest = classifier()
    
    X = forest_data.drop(columns=['id', target_resid, f'{target_resid}_above_threshold'])
    
    y_reg = forest_data[target_resid]
    y_class = forest_data[f'{target_resid}_above_threshold']
    
    reg_fit, reg_scores = fit_score_model(X, y_reg, nreg)
    bin_est, bin_scores = fit_score_model(X, y_class, nest, classification=True)
    
    all_col_imp, summed_imp = extract_feature_imp(bin_est, loan_feature_cols, X.columns)
    summed_imp = { feature: score for feature, score in sorted(summed_imp.items(), key=lambda item: item[1], reverse=True)}
    
    return {
        "partial_out_model": res_est,
        "residual_regressor": reg_fit,
        "residual_classifier": bin_est,
        "regression_scores": reg_scores,
        "classifier_scores": bin_scores,
        "pre_scale_target_stats": pre_scale_target_desc,
        "summed_importances": summed_imp,
        "all_importances": all_col_imp,
        "residual_df": forest_data
    }

In [ ]:
[col for col in data.columns if "edu" in col]

In [ ]:
edu_target = 'edu_aner_lag-5'

edu_to_lag = {
    'edu_aner': -5,
    'edu_share_gov_exp': -5,
    'edu_pupil_teacher': -5,
    'young_population': -5,
    'cash_surplus_deficit': -5, 
    'inflation': -5,
    'trade_share_gdp': -5,
    'freedom_house': -5
}

edu_observed_X_cols = [f"{obs_col}_lag-5" for obs_col in edu_to_lag.keys() if obs_col != "edu_aner"]

edu_results = end_to_end_project_eval(
    data, "edu", "edu_aner_lag-5", edu_to_lag,
    observed_X_cols=edu_observed_X_cols, 
    loan_feature_cols=probe_feature_cols,
    inverted_outcome=True
)

print("Results for education check: ", edu_results["classifier_scores"])
print({ key: round(value, 3) for key, value in edu_results["summed_importances"].items() })

```qui regress mortality_under5 pc_commit_health lag_mortality_under5 ///
            lag_gdp_pc_ppp lag_fertility lag_population ///
            lag_physicians_rate  lag_female_adult_literacy ///
            lag_access_water lag_access_sanitation ///
            hiv_prevalence conflict i.period i.nregionname, r```

In [ ]:
consolidated_df = pd.concat((health_results["residual_df"], edu_results["residual_df"]))

In [ ]:
consolidated_df.shape

In [ ]:
[col for col in consolidated_df.columns if "residual" in col]

In [ ]:
consolidated_df = consolidated_df.fillna(0)

In [ ]:
X = consolidated_df.drop(columns=['id', "residual_target", f'residual_target_above_threshold'])

y_reg = consolidated_df["residual_target"]
y_class = consolidated_df['residual_target_above_threshold']

reg_fit, reg_scores = fit_score_model(X, y_reg, RandomForestRegressor())
bin_est, bin_scores = fit_score_model(X, y_class, RandomForestClassifier(), classification=True)

all_col_imp, summed_imp = extract_feature_imp(bin_est, probe_feature_cols, X.columns)
summed_imp = { feature: score for feature, score in sorted(summed_imp.items(), key=lambda item: item[1], reverse=True)}

bin_scores

In [ ]:
summed_imp

## Build better models / 

In [ ]:
# try: SVC, Lasso, and a few others

In [ ]:
from sklearn.svm import SVC

In [ ]:
from xgboost import XGBClassifier

In [ ]:
bin_est, bin_scores = fit_score_model(X, y_class, SVC(probability=True), classification=True)

# all_col_imp, summed_imp = extract_feature_imp(bin_est, probe_feature_cols, X.columns)
# summed_imp = { feature: score for feature, score in sorted(summed_imp.items(), key=lambda item: item[1], reverse=True)}

bin_scores["roc_auc"]

In [ ]:
bin_est, bin_scores = fit_score_model(X, y_class, XGBClassifier(), classification=True)

bin_scores["roc_auc"]

## Extract SHAP and other values